In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

In [3]:
!find data -type f -name '*.csv'

data/swpc/kpindices-2022.csv
data/swpc/kpindices-2023.csv
data/dscovr/dsc_fc_summed_spectra_2022_v01.csv
data/dscovr/dsc_fc_summed_spectra_2023_v01.csv


In [4]:
x_df = pd.read_csv("./data/dscovr/dsc_fc_summed_spectra_2022_v01.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
y_df = pd.read_csv("./data/swpc/kpindices-2022.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = 0)

In [5]:
x_df.describe()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
count,524450.000000,524450.000000,524450.000000,511816.000000,511811.000000,511809.000000,511809.000000,511815.000000,511815.000000,511814.000000,...,20115.000000,11191.000000,8416.000000,3971.000000,3859.000000,213.000000,139.000000,44.000000,40.000000,35.000000
mean,0.098939,-0.219225,0.060599,58.218162,9.514369,63.246343,56.563212,74.062121,65.428809,94.261894,...,384.711976,332.966637,388.589832,326.915073,294.150454,390.130451,384.123885,376.326977,403.068593,367.789749
std,3.943064,4.535931,3.667523,38.506281,17.031107,44.696797,45.047853,60.528255,82.509457,114.361904,...,40.774888,48.215582,38.280578,57.801942,85.771695,52.754546,51.583450,69.893480,124.156716,143.007012
min,-16.712300,-26.376500,-23.060000,0.231726,0.231726,0.231726,0.231726,0.231726,0.231726,0.231726,...,214.319000,188.544000,211.826000,205.870000,108.950000,202.247000,112.290000,187.778000,0.231726,89.996200
25%,-3.000140,-3.305092,-1.967988,28.843000,0.231726,32.352500,27.916000,37.301550,25.640400,36.973425,...,365.230000,302.532500,367.203750,274.956000,211.853000,359.223000,352.946500,354.073000,366.691750,241.622000
50%,0.212469,-0.288369,0.034746,51.989300,0.386813,55.570600,46.977600,61.355300,45.470200,68.838050,...,383.495000,326.713000,393.807500,323.581000,309.980000,389.128000,395.347000,383.129500,427.067000,403.663000
75%,3.164427,3.017987,2.039108,84.737900,15.812800,90.033900,80.592700,100.403000,84.142500,114.472000,...,404.069000,363.782500,409.080750,377.611000,367.034500,416.967000,410.484000,411.971250,471.866000,449.118500
max,19.725300,22.834700,27.905900,415.389000,385.676000,493.748000,746.807000,1136.670000,1562.550000,1804.560000,...,662.013000,735.132000,772.122000,521.745000,541.136000,637.731000,568.053000,528.470000,655.892000,646.111000


In [6]:
y_df.head()

,date,3,6,9,12,15,18,21,24
0,2022-04-01,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0
1,2022-04-02,4.0,5.0,4.0,2.0,2.0,3.0,4.0,3.0
2,2022-04-03,4.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0
3,2022-04-04,4.0,4.0,3.0,2.0,2.0,2.0,2.0,1.0
4,2022-04-05,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0


We can see that the timestamp is different between data sources and we will need to fix that. Because we want to train on a relative measure of time we will convert the data into training examples with an integer ordinal number.

What other changes will be needed? 

- convert date and time to an ordinal count of seconds since the start time of one example
- convert the kp indices to an integer
- transform dscovr NaNs into 0 (for now - maybe there is a better value to use) 
- filter kp rows that have invalid data (-1s)
- normalize dscovr mag field values
- normalize dscovr solar wind values

The last transform we need to make is to organize the data from dscovr into training examples having a shape of (4320, 53, 2). In axis 0 we have 4,320 minutes per 3 days.
In axis 1 we have the 53 data points for each minute.
In axis 2 we have the ISO date timestamp of the Kp index we are learning and the Kp index we want to learn to predict.

Note that we will generate a new training example for every 3 hour Kp index for each day so we want to cache the pr with evious data and shift it for efficiency.

We need to save our training set to files with random training/valid/test splits. We will split 10% of the examples for validation and save them as 'valid.h5' and the remaining 90% will be our training examples stored as 'train.h5'.which is

We still don't have a held-out test set for evaluating the performance of our model. We will use the 2022 data for training and validation sets and the 2023 data as our test set. Of course, you could come back and change this by combining 2022 and 2023 into one big set then splitting it 3 ways, maybe with an 80/10/10 split which is commonly used.



First we will remove any rows where the timestamp is missing

In [7]:
x_df = x_df.drop(x_df[x_df[1].isna()].index)
print(x_df.shape)

(524450, 54)


In [8]:
print(y_df.shape)
y_df = y_df.drop(y_df[y_df['date'].isna()].index)
print(y_df.shape)

(365, 9)
(365, 9)


Now we will remove any rows where all of the solar wind values are missing (NaN)

In [9]:
x_df = x_df.drop(x_df[x_df.loc[:,4:].isnull().all(1)].index)
print(x_df.shape)

(511911, 54)


Now let's replace any remaining NaNs with 0

In [10]:
print(x_df.shape)
values = {}
for i in range(4, 54):
  values[i] = 0
print(values)
x_df = x_df.fillna(values)
print(x_df.shape)

(511911, 54)
{4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0}
(511911, 54)


In [11]:
x_df.describe()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
count,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,...,511911.000000,511911.000000,511911.000000,511911.000000,511911.00000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000
mean,0.115983,-0.245809,0.071499,58.207358,9.512510,63.233741,56.551942,74.048232,65.416539,94.244032,...,15.116849,7.279058,6.388556,2.535948,2.21743,0.162329,0.104302,0.032346,0.031495,0.025146
std,3.950377,4.545212,3.676730,38.510873,17.029962,44.701258,45.050440,60.531076,82.506584,114.358428,...,75.182786,49.208948,49.656902,29.129386,26.51012,8.028428,6.385241,3.547122,3.723996,3.256721
min,-16.712300,-26.376500,-23.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-2.981530,-3.328865,-1.954055,28.829400,0.231726,32.337400,27.899950,37.288800,25.625400,36.957850,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.237204,-0.323418,0.042795,51.979700,0.386135,55.553800,46.967800,61.336900,45.463100,68.817200,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.177780,2.994075,2.049095,84.733700,15.809700,90.025650,80.582100,100.397500,84.129800,114.466000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
max,19.725300,22.834700,27.905900,415.389000,385.676000,493.748000,746.807000,1136.670000,1562.550000,1804.560000,...,662.013000,735.132000,772.122000,521.745000,541.13600,637.731000,568.053000,528.470000,655.892000,646.111000


In [12]:
M = y_df.shape[0]*(y_df.shape[1]-1)
print(M, 'Kp indices')
print(4320*x_df.shape[1], 'DSCOVR data points for each Kp index')

2920 Kp indices
233280 DSCOVR data points for each Kp index


In [13]:
x_M = int(x_df.shape[0] / 4320)
print(y_df.shape)

(365, 9)


In [14]:
from sklearn.model_selection import train_test_split

train_ydf, valid_ydf = train_test_split(y_df, test_size=0.1)

In [15]:
train_M = train_ydf.shape[0] * (train_ydf.shape[1]-1)
valid_M = valid_ydf.shape[0] * (valid_ydf.shape[1]-1)
print(train_M, ' training examples and ', valid_M, ' validation examples')

2624  training examples and  296  validation examples


In [16]:
def get_X(t, indf):
  t0 = pd.to_datetime(t) - timedelta(days=3)
  match = indf.loc[:,0]==t0
  if indf[match].shape[0] == 1:
    x_idx = indf[match].index[0]
  else:
    x_idx = -1
    for h in range(24):
      nt = pd.to_datetime(t0) + timedelta(hours=h)
      match = indf.loc[:,0]==nt
      if indf[match].shape[0] == 1:
        x_idx = indf[match].index[0]
        break
  return x_idx, x_idx+4320

Check that get_X performs as expected

In [17]:
t = valid_ydf.iloc[0,0]
begin, end = get_X(t, x_df)
print('Start at ',x_df.loc[begin,0])
print('End   at ',x_df.loc[end,0])

Start at  2022-12-10 00:00:00
End   at  2022-12-13 00:00:00


Now step through each validation example and construct one training example with 4320 one minute (3 days) solar wind samples 

In [18]:
valid_X = np.zeros((valid_M, 4320, 53))
valid_Y = np.zeros((valid_M, 1))
n_example = 0
for m in range(valid_M):
  y_0 = int(m / 8)
  y_1 = m % 8
  print(m, y_0, y_1, valid_ydf.shape, t)
  t = valid_ydf.iloc[y_0, 0]
  seq = get_X(t, x_df)
  # check for exactly 3 days of data
  print(seq)
  if seq[1] < x_df.shape[0]:
    print(m, 'Start at ',x_df.iloc[seq[0],0], ' for ', x_df.iloc[seq[1],0] - x_df.iloc[seq[0],0])
    days_3 = timedelta(days=3)
    if x_df.iloc[seq[1],0] - days_3 == x_df.iloc[seq[0],0]:
      valid_X[n_example] = x_df.iloc[seq[0]:seq[1],1:].to_numpy()
      valid_Y[n_example] = valid_ydf.iloc[y_0, y_1+1]
      if np.isnan(valid_Y[n_example]):
        print(n_example, 'NaN')
      else:
        print(n_example, 'Good')
        n_example += 1

print('Found', n_example, 'training examples')
valid_X = valid_X[:n_example]
valid_Y = valid_Y[:n_example]
valid_M = n_example

0 0 0 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
0 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
1 0 1 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
1 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
2 0 2 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
2 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
3 0 3 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
3 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
4 0 4 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
4 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
5 0 5 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
5 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
6 0 6 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
6 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
7 0 7 (37, 9) 2022-12-13 00:00:00
(493920, 498240)
7 Start at  2022-12-18 09:30:00  for  3 days 01:34:00
8 1 0 (37, 9) 2022-12-13 00:00:00
(384480, 388800)
8 Start at  2022-10-02 14:12:00  for  3 days 00:00:00
0 Good
9 1 1 (37, 9) 2022-09-28 00:00:00
(384480, 38880

(387360, 391680)
124 Start at  2022-10-04 14:12:00  for  3 days 00:00:00
34 Good
125 15 5 (37, 9) 2022-09-30 00:00:00
(387360, 391680)
125 Start at  2022-10-04 14:12:00  for  3 days 00:00:00
35 Good
126 15 6 (37, 9) 2022-09-30 00:00:00
(387360, 391680)
126 Start at  2022-10-04 14:12:00  for  3 days 00:00:00
36 Good
127 15 7 (37, 9) 2022-09-30 00:00:00
(387360, 391680)
127 Start at  2022-10-04 14:12:00  for  3 days 00:00:00
37 Good
128 16 0 (37, 9) 2022-09-30 00:00:00
(207360, 211680)
128 Start at  2022-05-30 00:01:00  for  3 days 00:00:00
38 Good
129 16 1 (37, 9) 2022-05-28 00:00:00
(207360, 211680)
129 Start at  2022-05-30 00:01:00  for  3 days 00:00:00
39 Good
130 16 2 (37, 9) 2022-05-28 00:00:00
(207360, 211680)
130 Start at  2022-05-30 00:01:00  for  3 days 00:00:00
40 Good
131 16 3 (37, 9) 2022-05-28 00:00:00
(207360, 211680)
131 Start at  2022-05-30 00:01:00  for  3 days 00:00:00
41 Good
132 16 4 (37, 9) 2022-05-28 00:00:00
(207360, 211680)
132 Start at  2022-05-30 00:01:00  for 

88 Good
244 30 4 (37, 9) 2022-05-06 00:00:00
(175680, 180000)
244 Start at  2022-05-08 00:01:00  for  3 days 00:00:00
89 Good
245 30 5 (37, 9) 2022-05-06 00:00:00
(175680, 180000)
245 Start at  2022-05-08 00:01:00  for  3 days 00:00:00
90 Good
246 30 6 (37, 9) 2022-05-06 00:00:00
(175680, 180000)
246 Start at  2022-05-08 00:01:00  for  3 days 00:00:00
91 Good
247 30 7 (37, 9) 2022-05-06 00:00:00
(175680, 180000)
247 Start at  2022-05-08 00:01:00  for  3 days 00:00:00
92 Good
248 31 0 (37, 9) 2022-05-06 00:00:00
(480960, 485280)
248 Start at  2022-12-09 07:55:00  for  3 days 00:04:00
249 31 1 (37, 9) 2022-12-04 00:00:00
(480960, 485280)
249 Start at  2022-12-09 07:55:00  for  3 days 00:04:00
250 31 2 (37, 9) 2022-12-04 00:00:00
(480960, 485280)
250 Start at  2022-12-09 07:55:00  for  3 days 00:04:00
251 31 3 (37, 9) 2022-12-04 00:00:00
(480960, 485280)
251 Start at  2022-12-09 07:55:00  for  3 days 00:04:00
252 31 4 (37, 9) 2022-12-04 00:00:00
(480960, 485280)
252 Start at  2022-12-09 0

In [19]:
train_X = np.zeros((train_M, 4320, 53))
train_Y = np.zeros((train_M, 1))
n_example = 0
for m in range(train_M):
  y_0 = int(m / 8)
  y_1 = m % 8
  t = train_ydf.iloc[y_0, 0]
  #print(m, y_0, y_1, train_ydf.shape, t)
  seq = get_X(t, x_df)
  # check for exactly 3 days of data
  #print(seq)
  if seq[1] < x_df.shape[0]:
    #print(m, 'Start at ',x_df.iloc[seq[0],0], ' for ', x_df.iloc[seq[1],0] - x_df.iloc[seq[0],0])
    days_3 = timedelta(days=3)
    if x_df.iloc[seq[1],0] - days_3 == x_df.iloc[seq[0],0]:
      train_X[n_example] = x_df.iloc[seq[0]:seq[1],1:].to_numpy()
      train_Y[n_example] = train_ydf.iloc[y_0, y_1+1]
      #if np.isnan(train_Y[n_example]):
      if train_Y[n_example].size > 0:
        print(n_example, 'Good')
        n_example += 1
      else:
        print(n_example, 'Nan')

print('Found', n_example, 'training examples')
train_X = train_X[:n_example]
train_Y = train_Y[:n_example]
train_M = n_example

0 Good
1 Good
2 Good
3 Good
4 Good
5 Good
6 Good
7 Good
8 Good
9 Good
10 Good
11 Good
12 Good
13 Good
14 Good
15 Good
16 Good
17 Good
18 Good
19 Good
20 Good
21 Good
22 Good
23 Good
24 Good
25 Good
26 Good
27 Good
28 Good
29 Good
30 Good
31 Good
32 Good
33 Good
34 Good
35 Good
36 Good
37 Good
38 Good
39 Good
40 Good
41 Good
42 Good
43 Good
44 Good
45 Good
46 Good
47 Good
48 Good
49 Good
50 Good
51 Good
52 Good
53 Good
54 Good
55 Good
56 Good
57 Good
58 Good
59 Good
60 Good
61 Good
62 Good
63 Good
64 Good
65 Good
66 Good
67 Good
68 Good
69 Good
70 Good
71 Good
72 Good
73 Good
74 Good
75 Good
76 Good
77 Good
78 Good
79 Good
80 Good
81 Good
82 Good
83 Good
84 Good
85 Good
86 Good
87 Good
88 Good
89 Good
90 Good
91 Good
92 Good
93 Good
94 Good
95 Good
96 Good
97 Good
98 Good
99 Good
100 Good
101 Good
102 Good
103 Good
104 Good
105 Good
106 Good
107 Good
108 Good
109 Good
110 Good
111 Good
112 Good
113 Good
114 Good
115 Good
116 Good
117 Good
118 Good
119 Good
120 Good
121 Good
122 Good
123

963 Good
964 Good
965 Good
966 Good
967 Good
968 Good
969 Good
970 Good
971 Good
972 Good
973 Good
974 Good
975 Good
976 Good
977 Good
978 Good
979 Good
980 Good
981 Good
982 Good
983 Good
984 Good
985 Good
986 Good
987 Good
988 Good
989 Good
990 Good
991 Good
992 Good
993 Good
994 Good
995 Good
996 Good
997 Good
998 Good
999 Good
1000 Good
1001 Good
1002 Good
1003 Good
1004 Good
1005 Good
1006 Good
1007 Good
1008 Good
1009 Good
1010 Good
1011 Good
1012 Good
1013 Good
1014 Good
1015 Good
Found 1016 training examples


In [20]:
nan=np.argwhere(np.isnan(train_Y))
nan=nan[:,0]
print(nan)
train_Y = np.delete(train_Y, nan, axis=0)
train_X = np.delete(train_X, nan, axis=0)

[  37   38   70  104  106  121  122  124  130  160  161  166  261  279
  286  293  294  357  358  369  406  407  469  480  482  483  484  486
  487  518  584  585  586  588  589  591  615  655  680  721  777  783
  788  798  805  821  822  829  836  838  839  886  926 1009]


In [21]:
nan=np.argwhere(np.isnan(valid_Y))
nan=nan[:,0]
print(nan)
valid_Y = np.delete(valid_Y, nan, axis=0)
valid_X = np.delete(valid_X, nan, axis=0)

[]


In [22]:
print(train_X.shape, train_Y.shape)
print(valid_X.shape, valid_Y.shape)

(962, 4320, 53) (962, 1)
(117, 4320, 53) (117, 1)


In [41]:
kp1=np.argwhere(train_Y==5)


(24, 2)


In [47]:
# naive data augmentation
# 3 copies of each kp==4
kp1=np.argwhere(train_Y==4)
kp1=kp1[:,0]
print(kp1)
for n in range(3):
  train_Y=np.append(train_Y, train_Y[kp1], axis=0)
  train_X=np.append(train_X, train_X[kp1], axis=0)
# 10 copies of each kp==5
kp1=np.argwhere(train_Y==5)
kp1=kp1[:,0]
print(kp1)
for n in range(10):
  train_Y=np.append(train_Y, train_Y[kp1], axis=0)
  train_X=np.append(train_X, train_X[kp1], axis=0)

[ 15  46  47  50  84  85 112 114 149 196 197 198 199 200 201 202 203 300
 319 320 328 330 395 396 400 418 420 426 433 440 467 475 478 482 507 510
 522 536 538 539 546 561 593 594 595 596 598 599 602 623 626 634 638 639
 658 668 675 687 701 703 706 709 710 770 773 813 815 819 849 875 876 881
 882 884 899 904 910 911 939 961]
[ 48  49  61  82  83 111 419 438 439 441 449 481 508 509 587 588 600 601
 640 669 674 702 705 814]


In [48]:
with open('train.dat', 'wb') as f:
    np.save(f, train_X)
    np.save(f, train_Y)

In [49]:
with open('valid.dat', 'wb') as f:
    np.save(f, valid_X)
    np.save(f, valid_Y)